## Menu Tree
check if the new menu same with the old menu
```python
class Node {
        String key;
        int value;
        boolean active;
        List<Node> children;
 }
```

```
      Existing tree                                                   
        a(1, T)                                                 
      /         \                                                 
    b(2, T)   c(3, T)                                   
  /       \           \                                          
d(4, T) e(5, T)      g(7, T)                       

            New tree
            a(1, T)
          /        \                                             
   b(2, T)         c(3, T)  
   /    |    \           \    
d(4, T) e(5, T) f(6, T)    g(7, F) 
```


In [ ]:

class Node:
    def __init__(self,key,value,active):
        self.key = key
        self.value = value
        self.active = active
        self.children = []

    def equals(self, node)-> bool:
        if not node:
            return False

        return self.key == node.key and self.value==node.value and self.active == node.active

    
class Solution:
    def getModifiedMenus(self, oldMenu:Node, newMenu:Node) -> int:
        if not oldMenu and not newMenu:
            return 0
        count = 0
        if not oldMenu or not newMenu or not oldMenu.equals(newMenu):
            count += 1
            
        oldChildren = self.getChildren(oldMenu)
        newChildren = self.getChildren(newMenu)

        for k,v in oldChildren.items():
            count += self.getModifiedMenus(v, newChildren.get(k,None))

        for k,v in newChildren.items():
            if k not in oldChildren:
                count += self.getModifiedMenus(None, v)
        
        return count


    def getChildren(self, menu:Node)->dict:
        if not menu:
            return {}
        res = {}
        for child in menu.children:
            res[child.key] = child

        return res

"""
 Existing tree                                        
         a(1, T)                                                
        /       \                                                          
     b(2, T)   c(3, T)                                               
    /     \           \                                                        
d(4, T) e(5, T)   f(6, T)  


"""
a = Node("a",1,True)
b = Node("b",2,True)
c = Node("c",3,True)
d = Node("d",4,True)
e = Node("e",5,True)
g = Node("g",7,True)
a.children.append(b)
a.children.append(c)
b.children.extend([d,e])
c.children.append(g)

a1 = Node("a",1,True)
b1 = Node("b",2,True)
c1 = Node("c",3,True)
d1 = Node("d",4,True)
e1 = Node("e",5,True)
f1 = Node("f",6, True)
g1 = Node("g",7,False)
a1.children.append(b1)
a1.children.append(c1)
b1.children.extend([d1,e1,f1])
c1.children.append(g1)

"""                                             
    New tree 
    a(1, T)
        \
        c(3, F)
            \
            f(66, T)
"""
a2 = Node("a",1,True)
c2 = Node("c",3,False)
g2 = Node("g",77,True)

sol = Solution()
print(sol.getModifiedMenus(a,a1))
print(sol.getModifiedMenus(a,a2))


2
5


## Five minutes intervals
DD上的每家店铺有自己的营业时间，以"mon 2:02 pm"或者"wed 00:13 am"这样的格式来显示开门和关门时间。

店铺营业期间，DD会每隔5分钟给外卖小哥发消息，提醒可能会出现的订单；而且发消息时的分钟数一定是5的倍数。例如，店铺A营业期间，DD在xx:00会发一次通知；xx:05的时候再发一次；xx:10再发一次；以此类推。

DD会一直给外卖小哥发消息，直到店铺关门才停止。

以某一家店铺的开门时间为例，如果这家店铺开门时间的分钟数不是5的倍数，DD会选择一个离开门时间最近的时候来发送消息。例如，店铺A今天的开门时间是10:03 AM，DD会在10:05 AM的时候发出第一条消息（因为10:05比10:00距离开门时间更近）。 又比如说店铺A今天的开门时间是10:02 AM，DD会在10:00 AM发出第一条消息，即使店铺A实际上还没有开门。

关门时间同理：例如，店铺A今天的关门时间是11:03 PM，DD会在11:05 PM的时候发出最后一条消息，即使店铺A实际上已经关门。又比如说店铺A今天的关门时间是11:02 PM，DD会在11:00 PM发出最后一条消息。

现在希望能写一个function：
输入是一家店铺的开门时间，"xxx xx:xx xx", 和关门时间，"xxx xx:xx xx"；
要求输出一个List，里面是DD为这家店铺发出通知的所有时间（time intervals）。

例子1：
openTime = "mon 10:01 am", closeTime = "mon 10:10 am";
output = {"11000", "11005", "11010"}.

例子2:
openTime = "sat 11:01 pm", closeTime = "sat 11:10 pm";
output = {"62300", "62305", "62310"}.

In [19]:
class Clock:
    def __init__(self, timestamp:str):
        dayMap = {"mon":1,"tues":2,"wed":3,"thurs":4,"fri":5,"sat":6,"sun":7}
        timestamp = timestamp.strip()
        parts = timestamp.split(" ")
        self.day = dayMap[parts[0]]
        hourMinute = parts[1].split(":")

        self.hour = int(hourMinute[0])
        if parts[2] == "pm":
            self.hour = self.hour%12 + 12
        self.minute = int(hourMinute[1])
        self.minute = self.roundToNear5(self.minute)

    def roundToNear5(self,minute:int)->int:
        if minute %5 >=3:
            minute += (5- minute%5)
        else:
            minute -= minute%5
        return minute
    
    def addFiveMin(self):
        self.minute = (self.minute + 5) %60
        if self.minute == 0:
            self.hour = (self.hour+1) %24
            if self.hour == 0:
                self.day = (self.day + 1) % 7

    def isSame(self,openTime,endTime):
        return openTime.day == endTime.day and openTime.hour == endTime.hour and openTime.minute == endTime.minute
    
    def getCurrent(self):
        if self.hour < 10:
            hour = "0" + str(self.hour)
        else:
            hour = str(self.hour)
        
        if self.minute < 10:
            minute = "0" + str(self.minute)
        else:
            minute = str(self.minute)
        return "{}{}{}".format(self.day,hour,minute)

def five(opentime:str, closetime:str)->list:
    openClock = Clock(opentime)
    closeClock = Clock(closetime)
    res = []
    while not openClock.isSame(openClock, closeClock):
        res.append(openClock.getCurrent())
        openClock.addFiveMin()

    res.append(openClock.getCurrent())
    return res

# 例子1：
# openTime = "mon 10:01 am", closeTime = "mon 10:10 am";
# output = {"11000", "11005", "11010"}.

# 例子2:
# openTime = "sat 11:01 pm", closeTime = "sat 11:10 pm";
# output = {"62300", "62305", "62310"}.
print(five("  mon 10:01 am  ", "mon 10:10 am   "))
print(five("sat 11:01 pm", "sat 11:10 pm"))


['11000', '11005', '11010']
['62300', '62305', '62310']


## Remove Order
find the eligible number that greater than both left and right neighbor, remove it from list and find the next

everytime use the smallest one if there are multiple answers

ex

input = [3, 5, 1, 4, 2]

output = [4, 2, 5, 3, 1]

In [22]:
import heapq
def removeOrder(nums:list)->list:
    n = len(nums)
    left = [i-1 for i in range(n)]
    right = [i+1 for i in range(n)]
    left[0] = None
    right[-1] = None 
    removed = [False] *n

    heap = []
    for i in range(n):
        if isEligible(i,removed,nums,left,right):
            heapq.heappush(heap,(nums[i],i))

    res = []
    for _ in range(n):
        while True:
            val,i = heapq.heappop(heap)
            if removed[i] == False and isEligible(i,removed,nums,left,right):
                break
        res.append(val)
        removed[i] = True

        L,R = left[i],right[i]
        if L is not None:
            right[L] = R
        if R is not None:
            left[R] = L

        for nei in [L,R]:
            if nei is not None and removed[nei] == False and isEligible(nei,removed,nums,left,right):
                heapq.heappush(heap,(nums[nei],nei))

    return res

def isEligible(i,removed,nums,left,right):
    if removed[i]:
        return False
    val = nums[i]
    L = left[i]
    R = right[i]
    if L is not None and val <= nums[L]:
        return False
    if R is not None and val <= nums[R]:
        return False
    return True

test1 = [3,5,1,4,2]
test2 = [8,3,5,4,9,2]
print(removeOrder(test1))
print(removeOrder(test2))

[4, 2, 5, 3, 1]
[5, 8, 9, 4, 3, 2]
